<a href="https://colab.research.google.com/github/utopian18/Machine_Learning_in_Textile/blob/main/Weaving_dataset_4_random_search_cv_all_models_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.3 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#importing required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import lightgbm
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, ElasticNetCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import r2_score
import numpy as np

In [5]:
#imporing dataset
df = pd.read_csv('/content/drive/MyDrive/python 2023/weaving_rejection_dataset - original.csv')

In [6]:
df.head()

,Construction,Req_Finish_Fabrics,Fabric_Allowance,Rec_Beam_length(yds),Shrink_allow,Previous_pdn,Req_grey_fabric,Req_beam_length(yds),Total_Pdn(yds),Rejection,warp_count,weft_count,epi,ppi
0,40+40/2/40/110x80,31300.0,6.0,38286.9360,12.5,TOTAL,33297.872340,34797.651103,27646.0,285,double_40,80.0,110,80
1,40x40/110x90,10450.0,7.0,13057.5840,14.5,TOTAL,11236.559140,12017.350432,11019.0,39,40,40.0,110,90
2,40x40/110x80,900.0,11.5,1902.8640,12.5,TOTAL,1016.949153,1062.753843,1528.0,0,40,40.0,110,80
3,40x40/130x80,8000.0,8.0,10093.9280,14.0,TOTAL,8695.652174,9245.815159,8944.0,58,40,40.0,130,80
4,50x50/140x70,3500.0,8.1,69146.1408,9.1,TOTAL,3808.487486,3831.158751,59367.0,1043,50,50.0,140,70


In [7]:
#selecting the required columns
df= df[['Req_Finish_Fabrics', 'Fabric_Allowance','Rec_Beam_length(yds)', 'Shrink_allow', 'Req_grey_fabric',
    'Req_beam_length(yds)', 'warp_count',  'weft_count','epi', 'ppi', 'Rejection']]

In [8]:
#dependant and independent variables
X = df.copy()
y = X.pop('Rejection')

In [9]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [10]:
def label_encoding(x):
    def resultant_count(x1,x2):
        return int( (x1*x2)/(x1+x2) )
    if x == 'double_80':
        return resultant_count(80,80)
    elif x == 'double_40':
        return resultant_count(40,40)
    else:
        return int(x)

In [11]:
X_train['warp_count']= X_train['warp_count'].apply(lambda x: label_encoding(x))
X_test['warp_count'] = X_test['warp_count'].apply(lambda x: label_encoding(x))

In [12]:
# # scoring = make_scorer(r2_score)
# g_cv = GridSearchCV(DecisionTreeRegressor(random_state=0),
#               param_grid= dt_params, cv=5, refit=True)

# g_cv.fit(X_train, y_train)
# g_cv.best_params_

# result = g_cv.cv_results_
# # print(result)
# r2_score(y_test, g_cv.best_estimator_.predict(X_test))

In [13]:
# implemting Randomized Search CV
def calculate_gridcv(model,params, X_train, y_train,X_test, y_test, cv,n_iter):
    gdcv = RandomizedSearchCV(estimator=model,param_distributions=params, n_iter= n_iter, cv=cv, n_jobs=-1)
    gdcv.fit(X_train, y_train)
    best_model= gdcv.best_estimator_
    y_pred = gdcv.predict(X_test)
    score = r2_score(y_test, y_pred)
    best_estimator = gdcv.best_estimator_
    return [best_model, y_pred, score, best_estimator]


model_to_train = [RandomForestRegressor(), lightgbm.LGBMRegressor(), XGBRegressor(),
                  KNeighborsRegressor(), DecisionTreeRegressor() ]

grid_parameters = [
    {#random forest
        'n_estimators': [100, 200, 300, 400, 500],
'criterion': ['squared_error', 'absolute_error', 'friedman_mse', 'poisson'],
          'max_features':['sqrt', 'log2', None]},

  #Lightgbm
 {'boosting_type': ['gbdt','dart' ], 'num_leaves': [31, 50,60],
           'learning_rate':[0.1, 0.01],'n_estimators': [100, 200, 300],
           'importance_type': ['split', 'gain']},

{
    #XGboost
    'n_estimators':[50,100,200, 300,400 ],  'learning_rate':[0.1, 0.3, 0.5, 0.8] ,
          'booster': ['gbtree', 'gblinear', 'dart'],'max_depth': [1, 2,3],
         'sampling_method': ['uniform', 'gradient_based' ], 'gamma': [0, 0.01,0.5, 0.9 ], 'grow_policy' :['depthwise', 'lossguide']  },

    {#KNeighborsRegressor
        'n_neighbors': [5,10, 15,20], 'weights':['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']},

    { #Decison Tree
    'criterion':['squared_error'], 'splitter': ['best', 'random'],
    'min_samples_leaf': [1,2], 'min_weight_fraction_leaf': np.arange(0.0, 1.0, 0.1) , 'max_features': ['auto', 'sqrt', 'log2']
}]

In [14]:
model_pred_scores =[]
for i, model in enumerate(model_to_train):
    params = grid_parameters[i]
    results = calculate_gridcv(model, params, X_train, y_train, X_test, y_test, 5, 8)
    model_pred_scores.append(results)


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003035 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1270
[LightGBM] [Info] Number of data points in the train set: 17608, number of used features: 10
[LightGBM] [Info] Start training from score 55.570593


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
15 fits failed out of a total of 40.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 1247, in fit
    super().fit(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/tree/_classes.py", line 177, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _valida

In [16]:
for result in model_pred_scores:
    print(f'Model: {type(result[0]).__name__}, Score: {result[2]}, best_model:{result[3]}')

Model: RandomForestRegressor, Score: 0.9570022662435109, best_model:RandomForestRegressor(criterion='friedman_mse', max_features='log2',
                      n_estimators=300)
Model: LGBMRegressor, Score: 0.9435122712982089, best_model:LGBMRegressor(n_estimators=200)
Model: XGBRegressor, Score: 0.9462570961810853, best_model:XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.01, gpu_id=None, grow_policy='lossguide',
             importance_type=None, interaction_constraints=None,
             learning_rate=0.8, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
 

In [17]:
model_pred_scores

[[RandomForestRegressor(criterion='friedman_mse', max_features='log2',
                        n_estimators=300),
  array([  7.62803853,  11.69750003,  17.82066976, ...,  28.38247243,
          11.07355663, 112.19539167]),
  0.9570022662435109,
  RandomForestRegressor(criterion='friedman_mse', max_features='log2',
                        n_estimators=300)],
 [LGBMRegressor(n_estimators=200),
  array([ 23.59053842,  26.68581896,  11.72493579, ...,  33.89104689,
          13.4883884 , 161.71249979]),
  0.9435122712982089,
  LGBMRegressor(n_estimators=200)],
 [XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=0.01, gpu_id=None, grow_policy='lossguide',
               importance_type=None, interaction_constraints=None,
               learning_rate=0.8,